In [ ]:
!pip install spacy==2.3.5
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz
import re, nltk, spacy, gensim
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from bs4 import BeautifulSoup
from nltk.tokenize import ToktokTokenizer
from nltk.stem import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer 
from string import punctuation
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from keras.metrics import top_k_categorical_accuracy
from keras.optimizers import adam_v2
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Conv1D, MaxPooling1D, Flatten
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import pandas as pd
import string
import itertools
import io
import json

     |████████████████████████████████| 12.0 MB 9.5 MB/s 
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
path = "/content/drive/MyDrive/"
pubmed_database = path + "pubmed-diabetesme-set.txt"

#df = pd.read_csv(pubmed_database, sep=" -", header=None, 
                # names=["Terms", "Info"])

#display (df)

In [ ]:
def get_PMID_Title_Tabstract_and_mh(lines):
  s = "".join(lines)
  s = re.sub("\n\s+", "", s)
  s = s.strip()
  s = s.split("\n")

  title = {}
  abstract = ""
  mh = []

  for line in s:
    if line[0:5]=="TI  -":
      title = line[6:]
    elif line[0:5]=="AB  -":
      abstract = line[6:]
    elif line[0:5]=="MH  -":
      mh.append(line[6:])
  return [title,abstract, mh]

In [ ]:
records = []
for i, p in enumerate("pubmed-diabetesme-set.txt"):
  f = open(pubmed_database, "r")
  temp_record = []
  r = []
  for line in f:
    if line != "\n":
      r.append(line)
    else :
      temp_record.append(get_PMID_Title_Tabstract_and_mh(r))
      r = []
  f.close()
  records.append(temp_record)

In [ ]:
records[1][0]

["India is one of the epicentres of the global diabetes mellitus pandemic. Rapid socioeconomic development and demographic changes, along with increased susceptibility for Indian individuals, have led to the explosive increase in the prevalence of diabetes mellitus in India over the past four decades. Type 2 diabetes mellitus in Asian Indian people is characterized by a young age of onset and occurrence at low levels of BMI. Available data also suggest that the susceptibility of Asian Indian people to the complications of diabetes mellitus differs from that of white populations. Management of this disease in India faces multiple challenges, such as low levels of awareness, paucity of trained medical and paramedical staff and unaffordability of medications and services. Novel interventions using readily available resources and technology promise to revolutionise the care of patients with diabetes mellitus in India. As many of these challenges are common to most developing countries of t

In [ ]:
token = ToktokTokenizer()
stop_words = set(stopwords.words("english"))
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
punct = punctuation

def clean_text(text):
    text = re.sub("\d+", "", text)
    text = text.lower()
    text = re.sub(r"\'\n", " ", text)
    text = re.sub(r"\'\xa0", " ", text)
    text = re.sub('\s+', ' ', text) # matches all whitespace characters
    text = text.strip(' ')
    return text
  
def strip_list_noempty(mylist):
    
    newlist = (item.strip() if hasattr(item, 'strip') else item for item in mylist)
    return [item for item in newlist if item != '']
    
    
def clean_punct(text): 
    ''' Remove punctuations'''
    words = token.tokenize(text)
    punctuation_filtered = []
    regex = re.compile('[%s]' % re.escape(punct))
    remove_punctuation = str.maketrans(' ', ' ', punct)
    
    for w in words:
        w = re.sub('^[0-9]*', " ", w)
        punctuation_filtered.append(regex.sub('', w))
  
    filtered_list = strip_list_noempty(punctuation_filtered)
        
    return ' '.join(map(str, filtered_list))
  
def stopWordsRemove(text):
    words = token.tokenize(text)
    filtered = [w for w in words if not w in stop_words]
    
    return ' '.join(map(str, filtered))
  
def lemmatization(texts, allowed_postags, stop_words=stop_words):
    lemma = wordnet.WordNetLemmatizer()       
    doc = nlp(texts) 
    texts_out = []
    
    for token in doc:
        if token.pos_ in allowed_postags:
            
          if token.lemma_ not in ['-PRON-']:
              texts_out.append(token.lemma_)
              
          else:
              texts_out.append('')
     
    texts_out = ' '.join(texts_out)

    return texts_out

In [ ]:
for r in records:
  for d in r:
    d[0] = clean_text(d[0])
    d[0] = BeautifulSoup(d[0]).get_text()
    d[0] = clean_punct(d[0])
    d[0] = stopWordsRemove(d[0])

In [ ]:
records_combined = list(itertools.chain.from_iterable(records))

In [ ]:
def one_v_all_learning(records):
  X = []
  y = []
  for d in records:
    X.append(d[1])
    y.append(d[0])
  X = np.array(X)
  y = np.array(y)
  # Sampling dataset
  vectorizer_X = TfidfVectorizer(analyzer='word', min_df=0.0, max_df = 1.0, 
                                    strip_accents = None, encoding = 'utf-8', 
                                    preprocessor=None, 
                                    token_pattern=r"(?u)\S\S+", # Need to repeat token pattern
                                    max_features=1000)
  # 80/20 split
  X_train, X_test, y_train, y_test = train_test_split(
      X, y, test_size=0.2,train_size=0.8, random_state=0)

  # TF-IDF matrices
  X_tfidf_train = vectorizer_X.fit_transform(X_train)
  X_tfidf_test = vectorizer_X.transform(X_test)

  #dimensionality reduction dengan PCA
  pca = PCA(n_components=0.95)
  X_tfidf_train = pca.fit_transform(X_tfidf_train.todense())
  X_tfidf_test = pca.transform(X_tfidf_test.todense())

  mh_list = []
  for d in records:
    mh_list += d[1]
  values, counts = np.unique(mh_list, return_counts=True)
  ind = (-counts).argsort()[:10]
  top_tags = values[ind]
  print(top_tags)
  reports = {}
  for i in range(len(top_tags)):
    y_train_temp = [1 if top_tags[i] in row else 0 for row in y_train]
    y_test_temp = [1 if top_tags[i] in row else 0 for row in y_test]
    clf = LogisticRegression(penalty= 'elasticnet', solver= 'saga', l1_ratio=0.5, random_state=1)
    clf.fit(X_tfidf_train, y_train_temp)
    preds = clf.predict(X_tfidf_test)
    print("Hasil Klasifikasi Kelas {}".format(top_tags[i]))
    print(classification_report(y_test_temp, preds))
    reports[top_tags[i]] = classification_report(y_test_temp, preds, output_dict=True)
  return reports

In [ ]:
logreg_report_combined = one_v_all_learning(records_combined)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  import sys
